In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
import tikzplotlib
matplotlib.use("pgf")
plt.rcdefaults()
plt.style.use('seaborn')
plt.rcParams["font.family"] = "serif" # use serif/main font for text elements
plt.rcParams["text.usetex"] = True    # use inline math for ticks
plt.rcParams["pgf.rcfonts"] = False   # don't setup fonts from rc parameters
plt.rcParams["pgf.preamble"] = [
    "\\usepackage{newpxtext,newpxmath}",         # load additional packages
]

# plt.rcParams['text.usetex'] = True
# plt.rcParams['text.latex.preamble'] = r"\usepackage{newpxtext,newpxmath}"
plt.rcParams['lines.markeredgewidth'] = 1

# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['font.sans-serif'] = 'DejaVu Sans'
# plt.rcParams['font.monospace'] = 'Nimbus Mono L'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

import json
import re
from typing import Union

/var/folders/d3/t3g_ld9d3vbbs_hn07s09qzr0000gn/T/ipykernel_29265/1290617883.py:15: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  plt.rcParams["pgf.preamble"] = [


In [2]:
# Load results from JSON files
results_dir = "../results"

def libname_of_path(path):
    return re.search(".*lib(.*)\.so(.\d)?", path).group(1)


benchmarks = [
    'dlopen-dlclose',
    'dlsym',
    'function-call',
    'lazy-binding',
    # 'pngtest'
]

In [3]:
def load_benchmark_results(file_prefix: str, benchmark_type: Union[str, None] = None, print_iter_cnt: bool = False):
    
    if benchmark_type is None:
        benchmark_type = file_prefix

    with open(f"{results_dir}/{file_prefix}-baseline.json") as fp_baseline:
        res = dict()
        raw = json.load(fp_baseline)
        outer_iterations = raw[benchmark_type]['outer-iterations']
        inner_iterations = raw[benchmark_type]['inner-iterations']
        for lib in raw[benchmark_type]['libraries']:
            libname = libname_of_path(lib['library-path'])
            lib.pop('library-path')
            res[libname] = lib

        # We drop the first few iterations for each library because it will usually take longer due to paging in stuff etc.
        baseline = (pd.concat([pd.DataFrame(res[key]).assign(libname=key).set_index('libname').iloc[10:]
                              for key in res.keys()]) / inner_iterations
                   )
                   # ).assign(**{
                   #      'branch-mispredicts/branches(%)': lambda df: 100. * df.loc[:, 'branch-mispredicts'] / df.loc[:, 'branches']
                   # }).drop(columns=['branch-mispredicts'])

    with open(f"{results_dir}/{file_prefix}-sandbox.json") as fp_sandbox:
        res = dict()
        raw = json.load(fp_sandbox)
        assert raw[f'{benchmark_type}-sandbox']['outer-iterations'] == outer_iterations
        assert raw[f'{benchmark_type}-sandbox']['inner-iterations'] == inner_iterations
        for lib in raw[f'{benchmark_type}-sandbox']['libraries']:
            libname = libname_of_path(lib['library-path'])
            lib.pop('library-path')
            res[libname] = lib

        sandbox = (pd.concat([pd.DataFrame(res[key]).assign(libname=key).set_index('libname').iloc[10:]
                              for key in res.keys()]) / inner_iterations
                  )
                  # ).assign(**{
                  #      'branch-mispredicts/branches(%)': lambda df: 100. * df.loc[:, 'branch-mispredicts'] / df.loc[:, 'branches']
                  # }).drop(columns=['branch-mispredicts'])
        
    if print_iter_cnt:
        print(f"Outer iterations: {outer_iterations}, inner iteration: {inner_iterations}")
    
    return baseline, sandbox


In [62]:
def plot_benchmark_data(baseline: pd.DataFrame, sandbox: pd.DataFrame):
    
    libnames = set(baseline.index.values).intersection(sandbox.index.values)
    print(f"Libraries: {libnames}")
    counters = set(baseline.columns).intersection(sandbox.columns)
    print(f"Counters: {counters}")

    fig, axss = plt.subplots(nrows=len(libnames), ncols=len(counters), figsize=(10.75, 2.6*len(libnames)), sharey=True)
    plt.subplots_adjust(hspace=.4)
    # plt.subplots_adjust(wspace=.5)
    
    if len(libnames) > 1:
        # Hack for correct order in dlopen-dlclose
        libnames = sorted(libnames)
        tmp = libnames[-2]
        libnames[-2] = libnames[-1]
        libnames[-1] = tmp

    for rowindex, libname in enumerate(libnames):
        axs = axss[rowindex] if len(libnames) > 1 else axss
        # axs[0].set_title(libname)
        if len(libnames) > 1:
            axs[0].set_ylabel(r'\texttt{lib' + libname + r'}')
            
        # Hack the order of counters:
        counters = sorted(counters, reverse=True)
        counters = counters[1:] + [counters[0]]

        # for counter, ax in zip(sorted(counters, reverse=True), axs.ravel()):
        for counter, ax in zip(counters, axs.ravel()):
            vals, bins, patches = ax.hist(
                [baseline.loc[libname, counter].values, sandbox.loc[libname, counter].values],
                bins=15, histtype='stepfilled',
                color=['blue', 'orange'], alpha=.4,
                # label=[benchmark, f'{benchmark}-sandbox']
            )
            baseline_median = baseline.loc[libname, counter].median()
            sandbox_median = sandbox.loc[libname, counter].median()
            ax.axvline(x=baseline_median, color='blue', lw=.7, label=f"{baseline_median:,.1f}")
            ax.axvline(x=sandbox_median, color='orange', lw=.7, label=f"{sandbox_median:,.1f}")
            ax.set_xlim(0, None)
            ax.set_ylim(0, 1000)
            ax.set_xlabel(r"\texttt{" + counter + r"}")
            handles, labels = ax.get_legend_handles_labels()
            # handles.append(mpatches.Patch(color='white',
            #                               label=f"$\\%$ overhead: ${100 * (sandbox_median / baseline_median - 1):.2f}\\%$\\\\"
            #                                     "(median/median - $100\\%$)"))
            diff = np.abs(sandbox_median-baseline_median)
            _, xmax = ax.get_xlim()
            ax.annotate(f"${100 * (sandbox_median / baseline_median - 1):.2f}\\%$ overhead (${diff:.1f}$)",
                        xy=((sandbox_median+baseline_median)/2,1000),
                        xytext=((sandbox_median+baseline_median)/2,1020),
                        annotation_clip=False,
                        ha='center', va='bottom',
                        fontsize=9,
                        # was 7.36
                        arrowprops=dict(arrowstyle=f'-[, widthB={7.23*diff/xmax}, lengthB=.2', lw=0.8))
            ax.legend(handles=handles, title='Medians')
            
    fig.legend(handles=[mpatches.Patch(color='blue', label=benchmark),
                        mpatches.Patch(color='orange', label=f'{benchmark}-sandbox')],
               loc='right')

    # fig.tight_layout()

    # plt.show()
    plt.savefig(f"figs/{benchmark}.pdf", bbox_inches='tight')

In [63]:
for benchmark in benchmarks:
    print(f"==== {benchmark} ====")
    plot_benchmark_data(
        *load_benchmark_results(benchmark)
    )

==== dlopen-dlclose ====
Libraries: {'hello_world', 'z_nofio', 'png'}
Counters: {'INST_RETIRED', 'L1D_CACHE_REFILL', 'CPU_CYCLES', 'BR_MIS_PRED'}
==== dlsym ====
Libraries: {'hello_world_1000'}
Counters: {'INST_RETIRED', 'L1D_CACHE_REFILL', 'CPU_CYCLES', 'BR_MIS_PRED'}
==== function-call ====
Libraries: {'hello_world'}
Counters: {'INST_RETIRED', 'L1D_CACHE_REFILL', 'CPU_CYCLES', 'BR_MIS_PRED'}
==== lazy-binding ====
Libraries: {'hello_world_with_deps_10'}
Counters: {'INST_RETIRED', 'L1D_CACHE_REFILL', 'CPU_CYCLES', 'BR_MIS_PRED'}


In [6]:
def get_web_png_size_distribution():
    with open("../png_sizes.txt", "r") as f:
        widths, heights = [], []
        for png in f.readlines():
            m = re.search('(\\d+) x (\\d+)', png)
            if m is None:
                continue
            width, height = m.groups()
            widths.append(int(width))
            heights.append(int(height))
        widths, heights = np.array(widths), np.array(heights)
        return np.sqrt(widths * heights)
png_sizes = get_web_png_size_distribution()

In [7]:
counter = "CPU_CYCLES" if 1 else "INST_RETIRED"
mean_baseline, std_baseline, mean_sandbox, std_sandbox, mean_ratio, std_ratio, ratios = [], [], [], [], [], [], []
for i in 2**np.arange(0, 11):
    baseline, sandbox = load_benchmark_results(f"pngtest-parameterised-1000/pngtest-on-testpng{i}", "pngtest", print_iter_cnt=False)
    mean_baseline.append(np.mean(baseline.loc[:, counter]))
    std_baseline.append(np.std(baseline.loc[:, counter]))
    mean_sandbox.append(np.mean(sandbox.loc[:, counter]))
    std_sandbox.append(np.std(baseline.loc[:, counter]))
    ratio = sandbox / baseline
    ratios.append(ratio.loc[:, counter])
    mean_ratio.append(np.mean(ratio.loc[:, counter]))
    std_ratio.append(np.std(ratio.loc[:, counter]))

mean_baseline = np.array(mean_baseline)
std_baseline = np.array(std_baseline)
mean_sandbox = np.array(mean_sandbox)
std_sandbox = np.array(std_sandbox)
mean_ratio = np.array(mean_ratio)
std_ratio = np.array(std_ratio)
ratios = np.array(ratios)

# Overhead plot

fig, ax = plt.subplots(figsize=(8,8))

res = ax.boxplot(ratios.T - 1, bootstrap=10000, positions=2**np.arange(0, 11), widths=0.3*2**np.arange(0, 11), showfliers=True, flierprops={'marker': 'x', 'ms': 2})
boxplot_patch = res["boxes"][0]
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=0, symbol="$\%$", is_latex=True))
ax.axhline(y=0, dashes=(6,4), color="black", alpha=0.6)
ax.set_xscale('log')
ax.xaxis.set_major_formatter(mtick.LogFormatterSciNotation(base=2))
ax.xaxis.set_major_locator(mtick.LogLocator(base=2, subs=(1.0,)))
ax.xaxis.set_ticks(ticks=2**np.arange(0, 11))
ax.set_xlabel("PNG test image width (pixels)")
ax.set_ylabel(r"Overhead in \texttt{" + counter + r"}")

ax_hist = ax.twinx()
hist_patch = ax_hist.hist(png_sizes, bins=2**np.arange(0.5,12.5,1), density=False, histtype='stepfilled', color='orange', alpha=0.3)
num_pngs = png_sizes.shape[0]
ax_hist.set_yticks(ticks=num_pngs*np.arange(0, 0.3, 0.05), labels=["$0$", "$0.05$", "$0.1$", "$0.15$", "$0.2$", "$0.25$"], color="orange")
ax_hist.tick_params(which='major', length=7, color="darkorange")
# ax_hist.set_axis_off()
ax_hist.grid(False)
ax_hist.set_ylabel("Common PNG size distribution", color="darkorange")

# ax.legend([boxplot_patch, hist_patch[2][0]], ["Relative overhead", "Common PNG size distribution"], loc="lower left")

# ax.set_ybound(-0.05, 0.415)
# ax.legend()
# fig.tight_layout()
ax.set_title(r"Overhead of sandboxing in \texttt{pngtest} on different test images")

plt.savefig("figs/pngtest.pdf", bbox_inches='tight')


# Log-logscale absolute plot
fig, ax = plt.subplots(figsize=(8,6))

ax.errorbar(2**np.arange(0, 11), mean_baseline, yerr=2*std_baseline, fmt='.', ms=2, color='blue', label="baseline")
ax.errorbar(2**np.arange(0, 11), mean_sandbox,  yerr=2*std_sandbox,  fmt='.', ms=2, color='orange', label="sandbox")
ax.set_xscale("log")
ax.set_yscale("log")
ax.xaxis.set_major_formatter(mtick.LogFormatterSciNotation(base=2))
ax.xaxis.set_major_locator(mtick.LogLocator(base=2, subs=(1.0,)))
ax.xaxis.set_ticks(ticks=2**np.arange(0, 11))
ax.set_xlabel("PNG test image width (pixels)")
ax.set_ylabel(counter)
# ax.plot(2**np.arange(0, 11), mean_sandbox / mean_baseline, label="ratio")
# ax.plot(2**np.arange(0, 11), mean_ratio - 1, label=f"{counter} $\\pm 2\\sigma$")
# ax.fill_between(2**np.arange(0, 11), mean_ratio - 1 - 2*std_ratio, mean_ratio - 1 + 2*std_ratio, alpha=0.5)



plt.show()

/var/folders/d3/t3g_ld9d3vbbs_hn07s09qzr0000gn/T/ipykernel_29265/2924523823.py:74: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Ablation study

Root cause analysis, finding what overhead each operation in the function call trampoline is associated with.

In [8]:
ablation_stages = [
    "empty-call-trampoline",
    "clear-caller-saved-regs",
    "return-trampoline-retaddr-on-caller-stack",
    "trusted-stack",
    "clear-and-save-callee-saved-regs",
    "restricted-mode",
    "restrict-sp-fp-bounds",
]

In [9]:
dfs = [load_benchmark_results(f"ablation/ld-{ablation_stages[0]}.so.1", "function-call")[0].median().rename("baseline")]
for stage in ablation_stages:
    baseline, sandbox = load_benchmark_results(f"ablation/ld-{stage}.so.1", "function-call")
    dfs.append(sandbox.median().rename(stage))
ablation = pd.concat(dfs, axis="columns").transpose()
ablation

,CPU_CYCLES,BR_MIS_PRED,INST_RETIRED,L1D_CACHE_REFILL
baseline,15.34950,0.0035,19.3286,0.0005
empty-call-trampoline,21.42465,0.0034,21.3286,0.0008
clear-caller-saved-regs,22.35510,0.0032,26.3286,0.0007
return-trampoline-retaddr-on-caller-stack,29.47915,0.0031,29.3286,0.0007
trusted-stack,246.14520,0.0044,125.4774,0.0033
clear-and-save-callee-saved-regs,248.58225,0.0048,145.4775,0.0035
restricted-mode,252.08385,0.0046,149.4774,0.0035
restrict-sp-fp-bounds,251.59655,0.0047,159.4774,0.0035


In [10]:
fig, ax = plt.subplots(figsize=(12, 3))
fig.suptitle("Decomposing the function call overhead")

counters = ['CPU_CYCLES', 'INST_RETIRED']

ax.invert_yaxis()
ax.yaxis.grid(False)
# ax.xaxis.set_visible(False)
# ax.set_xlim(0, np.sum(data, axis=1).max())

ax.barh([r"\texttt{"+c+r"}" for c in counters], ablation.iloc[0].loc[counters], height=0.2, color='gainsboro', label="baseline")

for i in range(1, ablation.shape[0]):
    widths = ablation.diff().iloc[i].loc[counters]
    starts = ablation.iloc[i-1].loc[counters]
    ax.barh([r"\texttt{"+c+r"}" for c in counters], widths, left=starts, height=0.2,
            label=f"+{ablation.index[i]}")
    ax.set_xlim((0, 255))
    xcenters = starts + widths / 2

ax.legend(loc='lower right')

plt.savefig("figs/ablation.pdf", bbox_inches='tight')
# tikzplotlib.clean_figure()
# tikzplotlib.save('figs/ablation.tex')

### ImageMagick macro-benchmarks

In [11]:
def load_magick_data():
    with open("../results/magick10x.txt", "r") as f:
        filenames = []
        rw_baseline, rw_sandbox, rrsw_baseline, rrsw_sandbox, rrsiw_baseline, rrsiw_sandbox = [], [], [], [], [], []
        while True:
            line = f.readline()
            if not re.match(r'(\d+)/(\d+)\n', line):
                print(line)
                assert line == ""
                break
            filename = f.readline()[:-1]
            rwb = f.readline()[:-1]
            rws = f.readline()[:-1]
            rrswb = f.readline()[:-1]
            rrsws = f.readline()[:-1]
            rrsiwb = f.readline()[:-1]
            rrsiws = f.readline()[:-1]
            
            num_illegals = 0
            for s in [rwb, rws, rrswb, rrsws, rrsiwb, rrsiws]:
                if s.startswith("Illegal"):
                    num_illegals += 1
                    while f.readline().startswith("Illegal"):
                        pass
            
            if num_illegals > 0:
                continue
            
            filenames.append(filename)
            rw_baseline.append(int(rwb)/10)
            rw_sandbox.append(int(rws)/10)
            rrsw_baseline.append(int(rrswb)/10)
            rrsw_sandbox.append(int(rrsws)/10)
            rrsiw_baseline.append(int(rrsiwb)/10)
            rrsiw_sandbox.append(int(rrsiws)/10)
        
        return (pd.DataFrame({
                    'read-write': rw_baseline + rw_sandbox,
                    'read-resize-write': rrsw_baseline + rrsw_sandbox,
                    'read-resize-improved-write': rrsiw_baseline + rrsiw_sandbox
                },
                index=pd.MultiIndex.from_product((['baseline', 'sandbox'], filenames))
            ).swaplevel()
             .unstack()
             .swaplevel(axis='columns')
             .sort_index(axis='columns')
        )

magick = load_magick_data()

In [12]:
overhead = (100 * (magick.sandbox / magick.baseline - 1)).assign(overhead='overhead').set_index('overhead', append=True).unstack().swaplevel(axis='columns')
magick = pd.concat([magick, overhead], axis='columns')

In [13]:
# fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey='row')
fig, ax = plt.subplots(ncols=2, figsize=(9,5))
fig.suptitle("ImageMagick benchmarks")

ax[0].boxplot(magick.overhead.loc[:, 'read-write'], positions=[1], whis=[2.5,97.5], showfliers=True, bootstrap=10000, flierprops={'ms': 2, 'marker': 'x'}, capprops={'color': 'green'}, whiskerprops={'color': 'green'})
ax[0].boxplot(magick.overhead.loc[:, 'read-resize-write'], positions=[2], whis=[2.5,97.5], showfliers=True, bootstrap=10000, flierprops={'ms': 2, 'marker': 'x'}, capprops={'color': 'green'}, whiskerprops={'color': 'green'})
ax[0].boxplot(magick.overhead.loc[:, 'read-resize-improved-write'], positions=[3], whis=[2.5,97.5], showfliers=True, bootstrap=10000, flierprops={'ms': 2, 'marker': 'x'}, capprops={'color': 'green'}, whiskerprops={'color': 'green'})
ax[0].set_xticks(ticks=[1,2,3], labels=['read-write', 'read-resize-write', 'read-resize-improved-write'])
ax[0].xaxis.grid(False)
ax[0].yaxis.set_major_formatter(mtick.StrMethodFormatter(r'${x:.0f}\%$'))
# ax.set_yticks(ticks=[-0.4,-0.2,0,0.2,0.4], labels=[r'$-0.4\%$', r'$-0.2\%$', r'$0\%$', r'$0.2\%$', r'$0.4\%$'])
ax[0].set_ylabel(r"Wall-clock time overhead of sandboxed \texttt{libpng}")
ax[0].set_title(r"$2.5^\mathrm{th}$ and $97.5^\mathrm{th}$ percentiles, and outliers")

ax[1].boxplot(magick.overhead.loc[:, 'read-write'], positions=[1], whis=[5,95], showfliers=False, bootstrap=10000)
ax[1].boxplot(magick.overhead.loc[:, 'read-resize-write'], positions=[2], whis=[5,95], showfliers=False, bootstrap=10000)
ax[1].boxplot(magick.overhead.loc[:, 'read-resize-improved-write'], positions=[3], whis=[5,95], showfliers=False, bootstrap=10000)
ax[1].set_xticks(ticks=[1,2,3], labels=['read-write', 'read-resize-write', 'read-resize-improved-write'])
ax[1].xaxis.grid(False)
ax[1].yaxis.set_major_formatter(mtick.StrMethodFormatter(r'${x:.1f}\%$'))
ax[1].set_title(r"Medians, inter-quartile ranges, and $5^\mathrm{th}$ and $95^\mathrm{th}$ percentiles")

fig.tight_layout()
plt.savefig("figs/magick.pdf", bbox_inches='tight')

In [14]:
magick.overhead.loc[:, 'read-resize-improved-write'].median()

0.024183670144584113

In [15]:
def get_png_sizes():
    with open("../png_sizes.txt", "r") as f:
        filenames, widths, heights = [], [], []
        for png in f.readlines():
            m = re.search(r'/Users/gabor/Documents/dissertation/png-stats/pngs/(.*).png:\s*PNG image data, (\d+) x (\d+)', png)
            if m is None:
                continue
            filename, width, height = m.groups()
            filenames.append(f'{filename}.png')
            widths.append(int(width))
            heights.append(int(height))
        return (pd.DataFrame({'width': widths, 'height': heights}, index=filenames)
                .assign(size=lambda df: np.sqrt(df.width * df.height))
               )
df_png_sizes = get_png_sizes()

In [16]:
tmp = df_png_sizes.loc[:, ['size']].copy()
tmp.columns = pd.MultiIndex.from_product((tmp.columns, [None]))
magick = magick.merge(tmp, left_index=True, right_index=True)

In [17]:
bin_indices = np.digitize(magick.loc[:, ('size', None)], bins=2**np.arange(0.5,12.5,1))

fig, axs = plt.subplots(ncols=3, sharex='col', sharey='row', figsize=(12,2))

for workload, ax in zip(['read-write', 'read-resize-write', 'read-resize-improved-write'], axs.ravel()):
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(mtick.LogFormatterSciNotation(base=2))
    ax.xaxis.set_major_locator(mtick.LogLocator(base=2, subs=(1.0,)))
    ax.xaxis.set_ticks(ticks=2**np.arange(4, 12))

    for i in range(4,12):
        data = magick.overhead.loc[:, workload].loc[bin_indices == i]
        ax.boxplot(data, positions=[2**i], widths=[0.3*2**i], showfliers=False)